In [1]:
import cv2
import numpy as np
import os
import random
from tqdm import tqdm
from PIL import Image
import shutil
import zipfile

# ======================
# CONFIG
# ======================
OUT_DIR = "synthetic_shapes"
TOTAL_IMAGES = 2000
IMG_SIZE = 640

SHAPES = ["circle", "square", "triangle", "octagon", "star"]
COLORS = {
    "white": (255,255,255),
    "black": (0,0,0),
    "red":   (0,0,255),
    "green": (0,255,0),
    "blue":  (255,0,0)
}

# ======================
# CLEAN OLD DATASET
# ======================
if os.path.exists(OUT_DIR):
    shutil.rmtree(OUT_DIR)

    # remove zip if exists
if os.path.exists("synthetic_shapes.zip"):
    os.remove("synthetic_shapes.zip")

# ======================
# MAKE FOLDERS
# ======================
splits = {
    "train": int(TOTAL_IMAGES * 0.7),
    "val":   int(TOTAL_IMAGES * 0.2),
    "test":  TOTAL_IMAGES - int(TOTAL_IMAGES * 0.7) - int(TOTAL_IMAGES * 0.2)
}

os.makedirs(OUT_DIR, exist_ok=True)
for s in splits:
    os.makedirs(f"{OUT_DIR}/images/{s}", exist_ok=True)
    os.makedirs(f"{OUT_DIR}/labels/{s}", exist_ok=True)


# ======================
# Helper: draw shapes
# ======================
def draw_shape(shape, color, img):
    cx = random.randint(150, IMG_SIZE-150)
    cy = random.randint(150, IMG_SIZE-150)
    size = random.randint(60, 120)
    angle = random.randint(0, 360)

    if shape == "circle":
        cv2.circle(img, (cx, cy), size, color, -1)
        pts = np.array([[cx-size, cy-size], [cx+size, cy+size]])

    elif shape == "square":
        pts = cv2.boxPoints(((cx, cy), (size*2, size*2), angle)).astype(int)
        cv2.fillPoly(img, [pts], color)

    elif shape == "triangle":
        pts = np.array([
            [cx, cy-size],
            [cx-size, cy+size],
            [cx+size, cy+size]
        ])
        M = cv2.getRotationMatrix2D((cx,cy), angle,1.0)
        pts = cv2.transform(np.array([pts]), M)[0].astype(int)
        cv2.fillPoly(img, [pts], color)

    elif shape == "octagon":
        pts = []
        for i in range(8):
            theta = (np.pi/4 * i) + np.deg2rad(angle)
            pts.append([cx + size*np.cos(theta), cy + size*np.sin(theta)])
        pts = np.array(pts, dtype=np.int32)
        cv2.fillPoly(img, [pts], color)

    elif shape == "star":
        pts = []
        for i in range(10):
            r = size if i % 2 == 0 else size/2
            theta = (2*np.pi * i / 10) + np.deg2rad(angle)
            pts.append([cx + r*np.cos(theta), cy + r*np.sin(theta)])
        pts = np.array(pts, dtype=np.int32)
        cv2.fillPoly(img, [pts], color)

    x_min = np.min(pts[:,0])
    y_min = np.min(pts[:,1])
    x_max = np.max(pts[:,0])
    y_max = np.max(pts[:,1])

    x_min = max(0, x_min); y_min = max(0, y_min)
    x_max = min(IMG_SIZE, x_max); y_max = min(IMG_SIZE, y_max)

    return (x_min, y_min, x_max, y_max)


# ======================
# RANDOM BACKGROUND
# ======================
def random_background():
    img = np.random.randint(60, 255, (IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)
    if random.random() > 0.5:
        img = cv2.GaussianBlur(img, (11,11), 5)
    return img


# ======================
# MAIN LOOP — GENERATE DATASET
# ======================
for split, count in splits.items():
    print(f"\nGenerating {count} images for {split}")
    for i in tqdm(range(count)):

        img = random_background()

        shape = random.choice(SHAPES)
        color_name = random.choice(list(COLORS.keys()))
        color = COLORS[color_name]

        x1,y1,x2,y2 = draw_shape(shape, color, img)

        alpha = random.uniform(0.7, 1.4)
        beta = random.randint(-30, 30)
        img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

        fname = f"{split}_{i}.jpg"
        cv2.imwrite(f"{OUT_DIR}/images/{split}/{fname}", img)

        cls_id = SHAPES.index(shape)

        x_center = (x1+x2)/2/IMG_SIZE
        y_center = (y1+y2)/2/IMG_SIZE
        w = (x2-x1)/IMG_SIZE
        h = (y2-y1)/IMG_SIZE

        with open(f"{OUT_DIR}/labels/{split}/{fname.replace('.jpg','.txt')}", "w") as f:
            f.write(f"{cls_id} {x_center} {y_center} {w} {h}")

print("\nDataset generation completed!")

# ======================
# ZIP THE FOLDER
# ======================
zipf = zipfile.ZipFile("synthetic_shapes.zip", "w", zipfile.ZIP_DEFLATED)

for root, dirs, files in os.walk(OUT_DIR):
    for file in files:
        filepath = os.path.join(root, file)
        zipf.write(filepath, os.path.relpath(filepath, OUT_DIR))

zipf.close()
print("\nZIP file created: synthetic_shapes.zip")



Generating 1400 images for train


100%|██████████████████████████████████████████████████████████████████████████████| 1400/1400 [01:25<00:00, 16.38it/s]



Generating 400 images for val


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [00:24<00:00, 16.52it/s]



Generating 200 images for test


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:12<00:00, 16.63it/s]



Dataset generation completed!

ZIP file created: synthetic_shapes.zip


In [2]:
from IPython.display import FileLink
FileLink("synthetic_shapes.zip")


C:\Users\hp omen 4 pro\synthetic_shapes.zip

In [3]:
%%writefile synthetic_shapes.yaml
path: synthetic_shapes

train: images/train
val: images/val
test: images/test

names:
  0: circle
  1: square
  2: triangle
  3: octagon
  4: star


Writing synthetic_shapes.yaml


In [5]:
import os
os.path.abspath("synthetic_shapes")


'C:\\Users\\hp omen 4 pro\\synthetic_shapes'

In [6]:
%%writefile synthetic_shapes.yaml
path: "C:/Users/hp omen 4 pro/synthetic_shapes"

train: images/train
val: images/val
test: images/test

names:
  0: circle
  1: square
  2: triangle
  3: octagon
  4: star


Overwriting synthetic_shapes.yaml


In [7]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
model.train(
    data="synthetic_shapes.yaml",
    epochs=70,
    imgsz=640,
    batch=16
)


New https://pypi.org/project/ultralytics/8.3.231 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.58  Python-3.9.21 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=synthetic_shapes.yaml, epochs=70, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=

train: Scanning C:\Users\hp omen 4 pro\synthetic_shapes\labels\train... 1400 images, 0 backgrounds, 0 corrupt: 100%|███


train: New cache created: C:\Users\hp omen 4 pro\synthetic_shapes\labels\train.cache


val: Scanning C:\Users\hp omen 4 pro\synthetic_shapes\labels\val... 400 images, 0 backgrounds, 0 corrupt: 100%|████████


val: New cache created: C:\Users\hp omen 4 pro\synthetic_shapes\labels\val.cache
Plotting labels to runs\detect\train9\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train9
Starting training for 70 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      28/70      2.13G     0.2945      0.263     0.8721         18        640: 100%|██████████| 88/88 [00:36<00:00,  2.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:08

                   all        400        400      0.999          1      0.995      0.995



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      29/70      2.13G     0.2742     0.2787     0.8769         27        640:   1%|          | 1/88 [00:04<06:32,  4.5


RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [3]:
from ultralytics import YOLO

model = YOLO("runs/detect/train9/weights/best.pt")
results = model("11.bmp", save=True, device="cpu")   # CPU = safe, no CUDA crash



image 1/1 C:\Users\hp omen 4 pro\11.bmp: 544x640 1 circle, 1 square, 3 stars, 301.7ms
Speed: 14.0ms preprocess, 301.7ms inference, 3.0ms postprocess per image at shape (1, 3, 544, 640)
Results saved to runs\detect\predict2


In [5]:
import os
import urllib.request

os.makedirs("backgrounds", exist_ok=True)

urls = [
    "https://images.unsplash.com/photo-1519389950473-47ba0277781c",
    "https://images.unsplash.com/photo-1508780709619-79562169bc64",
    "https://images.unsplash.com/photo-1504384308090-c894fdcc538d",
    "https://images.unsplash.com/photo-1496307042754-b4aa456c4a2d",
    "https://images.unsplash.com/photo-1500530855697-b586d89ba3ee",
    "https://images.unsplash.com/photo-1501785888041-af3ef285b470"
]

for i, url in enumerate(urls):
    out = f"backgrounds/bg{i+1}.jpg"
    print("Downloading", out)
    urllib.request.urlretrieve(url, out)

print("All background images downloaded successfully!")


All background images downloaded successfully!


In [1]:
import cv2
import numpy as np
import os
import random
from tqdm import tqdm
import shutil
import zipfile
import glob

# ======================
# CONFIG
# ======================
OUT_DIR = "synthetic_shapes_v4"
TOTAL_IMAGES = 6000
IMG_SIZE = 640

SHAPES = ["circle", "square", "triangle", "octagon", "star"]
COLOR_NAMES = ["white", "black", "red", "green", "blue"]
COLORS = {
    "white": (255,255,255),
    "black": (0,0,0),
    "red":   (0,0,255),
    "green": (0,255,0),
    "blue":  (255,0,0)
}

# Create 25 class combinations
CLASSES = []
for s in SHAPES:
    for c in COLOR_NAMES:
        CLASSES.append(f"{s}_{c}")

BACKGROUND_FILES = glob.glob("backgrounds/*.jpg")

# ======================
# CLEAN OLD
# ======================
if os.path.exists(OUT_DIR):
    shutil.rmtree(OUT_DIR)

if os.path.exists("synthetic_shapes_v4.zip"):
    os.remove("synthetic_shapes_v4.zip")

splits = {
    "train": int(TOTAL_IMAGES * 0.7),
    "val":   int(TOTAL_IMAGES * 0.2),
    "test":  TOTAL_IMAGES - int(TOTAL_IMAGES * 0.7) - int(TOTAL_IMAGES * 0.2)
}

os.makedirs(OUT_DIR, exist_ok=True)
for s in splits:
    os.makedirs(f"{OUT_DIR}/images/{s}", exist_ok=True)
    os.makedirs(f"{OUT_DIR}/labels/{s}", exist_ok=True)


# ======================
# CAMERA TILT
# ======================
def camera_tilt(img):
    h, w = img.shape[:2]
    dx = random.randint(-35, 35)
    dy = random.randint(-35, 35)

    pts1 = np.float32([[0,0],[w,0],[0,h],[w,h]])
    pts2 = np.float32([
        [dx, dy],
        [w-dx, -dy],
        [-dx, h+dy],
        [w+dx, h-dy]
    ])
    M = cv2.getPerspectiveTransform(pts1, pts2)
    return cv2.warpPerspective(img, M, (w, h))


# ======================
# SHADOW
# ======================
def add_shadow(img, pts):
    overlay = img.copy()
    shadow = pts + np.array([random.randint(10, 25), random.randint(10, 25)])
    cv2.fillPoly(overlay, [shadow], (0,0,0))
    return cv2.addWeighted(overlay, 0.4, img, 0.6, 0)


# ======================
# REFLECTION / SPECULAR
# ======================
def add_highlight(img):
    overlay = img.copy()
    x = random.randint(0, IMG_SIZE)
    y = random.randint(0, IMG_SIZE)
    r = random.randint(60, 120)
    cv2.circle(overlay, (x,y), r, (255,255,255), -1)
    return cv2.addWeighted(overlay, 0.15, img, 0.85, 0)


# ======================
# BASLER CAMERA NOISE
# ======================
def add_camera_noise(img):
    noise = np.random.normal(0, 10, img.shape).astype(np.int16)
    img = img.astype(np.int16) + noise
    img = np.clip(img, 0, 255).astype(np.uint8)
    img = cv2.GaussianBlur(img, (5,5), random.uniform(0,1.2))
    return img


# ======================
# DRAW SHAPE (returns image + pts + bbox)
# ======================
def draw_shape(img, shape, color):
    cx = random.randint(120, IMG_SIZE-120)
    cy = random.randint(120, IMG_SIZE-120)
    size = random.randint(50, 120)
    angle = random.randint(0, 360)

    if shape == "circle":
        pts = np.array([
            [cx-size, cy-size],
            [cx+size, cy-size],
            [cx+size, cy+size],
            [cx-size, cy+size]
        ])
        cv2.circle(img, (cx, cy), size, color, -1)

    elif shape == "square":
        pts = cv2.boxPoints(((cx,cy),(size*2,size*2),angle)).astype(int)
        cv2.fillPoly(img, [pts], color)

    elif shape == "triangle":
        pts = np.array([
            [cx, cy-size],
            [cx-size, cy+size],
            [cx+size, cy+size]
        ])
        M = cv2.getRotationMatrix2D((cx,cy),angle,1.0)
        pts = cv2.transform(np.array([pts]),M)[0].astype(int)
        cv2.fillPoly(img, [pts], color)

    elif shape == "octagon":
        pts = []
        for i in range(8):
            θ = (np.pi/4 * i) + np.deg2rad(angle)
            pts.append([cx + size*np.cos(θ), cy + size*np.sin(θ)])
        pts = np.array(pts, np.int32)
        cv2.fillPoly(img, [pts], color)

    elif shape == "star":
        pts = []
        for i in range(10):
            r = size if i % 2 == 0 else size/2
            θ = (2*np.pi * i / 10) + np.deg2rad(angle)
            pts.append([cx + r*np.cos(θ), cy + r*np.sin(θ)])
        pts = np.array(pts, np.int32)
        cv2.fillPoly(img, [pts], color)

    # Add shadow
    img = add_shadow(img, pts)

    # BBOX
    x_min = max(0, np.min(pts[:,0]))
    y_min = max(0, np.min(pts[:,1]))
    x_max = min(IMG_SIZE, np.max(pts[:,0]))
    y_max = min(IMG_SIZE, np.max(pts[:,1]))

    return img, pts, (x_min, y_min, x_max, y_max)


# ======================
# LOAD BACKGROUND
# ======================
def load_background():
    bg = cv2.imread(random.choice(BACKGROUND_FILES))
    bg = cv2.resize(bg, (IMG_SIZE, IMG_SIZE))
    return bg


# ======================
# MAIN GENERATOR
# ======================
for split, count in splits.items():
    print(f"Generating {count} {split} images...")
    for i in tqdm(range(count)):

        img = load_background()
        labels = []

        # random number of objects
        n_objects = random.randint(2,6)

        for _ in range(n_objects):
            shape = random.choice(SHAPES)
            color_name = random.choice(COLOR_NAMES)
            color = COLORS[color_name]

            img, pts, (x1,y1,x2,y2) = draw_shape(img, shape, color)

            class_name = f"{shape}_{color_name}"
            cls_id = CLASSES.index(class_name)

            x_center = (x1+x2)/2/IMG_SIZE
            y_center = (y1+y2)/2/IMG_SIZE
            w = (x2-x1)/IMG_SIZE
            h = (y2-y1)/IMG_SIZE

            labels.append(f"{cls_id} {x_center} {y_center} {w} {h}")

        # CAMERA TILT
        img = camera_tilt(img)

        # REFLECTION
        if random.random() > 0.6:
            img = add_highlight(img)

        # BASLER NOISE
        img = add_camera_noise(img)

        fname = f"{split}_{i}.jpg"
        cv2.imwrite(f"{OUT_DIR}/images/{split}/{fname}", img)

        with open(f"{OUT_DIR}/labels/{split}/{fname.replace('.jpg','.txt')}", "w") as f:
            f.write("\n".join(labels))


# ======================
# ZIP OUTPUT
# ======================
zipf = zipfile.ZipFile("synthetic_shapes_v4.zip", "w")
for root, dirs, files in os.walk(OUT_DIR):
    for file in files:
        zipf.write(os.path.join(root, file))
zipf.close()

print("Dataset v4 Ready!")


Generating 4200 train images...


100%|████████████████████████████████████████████████████████████████████████████| 4200/4200 [1:34:17<00:00,  1.35s/it]


Generating 1200 val images...


100%|██████████████████████████████████████████████████████████████████████████████| 1200/1200 [26:39<00:00,  1.33s/it]


Generating 600 test images...


100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [13:22<00:00,  1.34s/it]


Dataset v4 Ready!


In [11]:
import cv2
import numpy as np
import os
import random
from tqdm import tqdm
import shutil
import zipfile
import glob

# ======================
# CONFIG
# ======================
OUT_DIR = "synthetic_shapes_v5"
TOTAL_IMAGES = 6000
IMG_SIZE = 640

SHAPES = ["circle", "square", "triangle", "octagon", "star"]
COLOR_NAMES = ["white", "black", "red", "green", "blue"]
COLORS = {
    "white": (255,255,255),
    "black": (0,0,0),
    "red":   (0,0,255),
    "green": (0,255,0),
    "blue":  (255,0,0)
}

# 25 classes (shape + color)
CLASSES = [f"{s}_{c}" for s in SHAPES for c in COLOR_NAMES]

BACKGROUND_FILES = glob.glob("backgrounds/*.jpg")
if len(BACKGROUND_FILES) == 0:
    raise Exception("No backgrounds found in /backgrounds folder")


# ======================
# CLEAN OLD DATASET
# ======================
if os.path.exists(OUT_DIR):
    shutil.rmtree(OUT_DIR)

if os.path.exists("synthetic_shapes_v5.zip"):
    os.remove("synthetic_shapes_v5.zip")

splits = {
    "train": int(TOTAL_IMAGES * 0.7),
    "val":   int(TOTAL_IMAGES * 0.2),
    "test":  TOTAL_IMAGES - int(TOTAL_IMAGES * 0.7) - int(TOTAL_IMAGES * 0.2)
}

for s in splits:
    os.makedirs(f"{OUT_DIR}/images/{s}", exist_ok=True)
    os.makedirs(f"{OUT_DIR}/labels/{s}", exist_ok=True)


# ======================
# LOAD BACKGROUND (TABLE LOOK)
# ======================
def load_background():
    bg = cv2.imread(random.choice(BACKGROUND_FILES))
    bg = cv2.resize(bg, (IMG_SIZE, IMG_SIZE))

    # Darken top region to look like depth / wall
    table_border = int(IMG_SIZE * 0.35)
    overlay = bg.copy()
    overlay[:table_border] = (overlay[:table_border] * 0.6).astype(np.uint8)

    bg = cv2.addWeighted(overlay, 0.4, bg, 0.6, 0)
    return bg


# ======================
# CAMERA TILT (REALISTIC TOP VIEW)
# ======================
def camera_tilt(img):
    h, w = img.shape[:2]
    dx = random.randint(-8, 8)
    dy = random.randint(-8, 8)

    pts1 = np.float32([[0,0],[w,0],[0,h],[w,h]])
    pts2 = np.float32([
        [dx, dy],
        [w-dx, -dy],
        [-dx, h+dy],
        [w+dx, h-dy]
    ])
    M = cv2.getPerspectiveTransform(pts1, pts2)
    return cv2.warpPerspective(img, M, (w, h))


# ======================
# SOFT TABLE SHADOW
# ======================
def add_shadow(img, pts):
    overlay = img.copy()
    shift = random.randint(5, 15)
    shadow = pts + np.array([shift, shift])

    cv2.fillPoly(overlay, [shadow], (25,25,25))
    return cv2.addWeighted(overlay, 0.35, img, 0.65, 0)


# ======================
# REFLECTION HIGHLIGHT
# ======================
def add_highlight(img):
    overlay = img.copy()
    x = random.randint(0, IMG_SIZE)
    y = random.randint(int(IMG_SIZE*0.35), IMG_SIZE)
    r = random.randint(40, 110)
    cv2.circle(overlay, (x,y), r, (255,255,255), -1)
    return cv2.addWeighted(overlay, 0.10, img, 0.90, 0)


# ======================
# BASLER CAMERA NOISE
# ======================
def add_camera_noise(img):
    noise = np.random.normal(0, 10, img.shape).astype(np.int16)
    img = img.astype(np.int16) + noise
    img = np.clip(img, 0, 255).astype(np.uint8)
    img = cv2.GaussianBlur(img, (5,5), random.uniform(0, 1.2))
    return img


# ======================
# DRAW SHAPE (FLAT ON TABLE)
# ======================
def draw_shape(img, shape, color):
    # Force objects to lie on table (lower 65% area)
    table_min = int(IMG_SIZE * 0.35)
    table_max = IMG_SIZE - 40

    cx = random.randint(120, IMG_SIZE - 120)
    cy = random.randint(table_min, table_max)

    size = random.randint(55, 120)
    angle = random.randint(0, 360)

    if shape == "circle":
        pts = np.array([
            [cx-size, cy-size],
            [cx+size, cy-size],
            [cx+size, cy+size],
            [cx-size, cy+size]
        ])
        cv2.circle(img, (cx, cy), size, color, -1)

    elif shape == "square":
        pts = cv2.boxPoints(((cx, cy), (size*2, size*2), angle)).astype(int)
        cv2.fillPoly(img, [pts], color)

    elif shape == "triangle":
        pts = np.array([
            [cx, cy-size],
            [cx-size, cy+size],
            [cx+size, cy+size]
        ])
        M = cv2.getRotationMatrix2D((cx,cy), angle, 1.0)
        pts = cv2.transform(np.array([pts]), M)[0].astype(int)
        cv2.fillPoly(img, [pts], color)

    elif shape == "octagon":
        pts = []
        for i in range(8):
            t = (np.pi/4 * i) + np.deg2rad(angle)
            pts.append([cx + size*np.cos(t), cy + size*np.sin(t)])
        pts = np.array(pts, np.int32)
        cv2.fillPoly(img, [pts], color)

    elif shape == "star":
        pts = []
        for i in range(10):
            r = size if i % 2 == 0 else size/2
            t = (2*np.pi * i / 10) + np.deg2rad(angle)
            pts.append([cx + r*np.cos(t), cy + r*np.sin(t)])
        pts = np.array(pts, np.int32)
        cv2.fillPoly(img, [pts], color)

    # Add realistic table shadow
    img = add_shadow(img, pts)

    # bounding box
    x_min = max(0, np.min(pts[:,0]))
    y_min = max(0, np.min(pts[:,1]))
    x_max = min(IMG_SIZE, np.max(pts[:,0]))
    y_max = min(IMG_SIZE, np.max(pts[:,1]))

    return img, pts, (x_min, y_min, x_max, y_max)


# ======================
# MAIN GENERATOR
# ======================
for split, count in splits.items():
    print(f"Generating {count} {split} images...")

    for i in tqdm(range(count)):
        img = load_background()
        labels = []

        # random number of table objects
        n_objects = random.randint(2, 6)

        for _ in range(n_objects):
            shape = random.choice(SHAPES)
            color_name = random.choice(COLOR_NAMES)
            color = COLORS[color_name]

            img, pts, (x1,y1,x2,y2) = draw_shape(img, shape, color)

            class_name = f"{shape}_{color_name}"
            cls_id = CLASSES.index(class_name)

            xc = (x1+x2)/2/IMG_SIZE
            yc = (y1+y2)/2/IMG_SIZE
            w = (x2-x1)/IMG_SIZE
            h = (y2-y1)/IMG_SIZE

            labels.append(f"{cls_id} {xc} {yc} {w} {h}")

        # Add realism
        img = camera_tilt(img)
        if random.random() > 0.6:
            img = add_highlight(img)
        img = add_camera_noise(img)

        # Save file
        fname = f"{split}_{i}.jpg"
        cv2.imwrite(f"{OUT_DIR}/images/{split}/{fname}", img)

        with open(f"{OUT_DIR}/labels/{split}/{fname.replace('.jpg','.txt')}", "w") as f:
            f.write("\n".join(labels))


# ======================
# ZIP OUTPUT
# ======================
zipf = zipfile.ZipFile("synthetic_shapes_v5.zip", "w")
for root, dirs, files in os.walk(OUT_DIR):
    for file in files:
        zipf.write(os.path.join(root, file))
zipf.close()

print("Dataset v5 Ready! (Realistic Table Version)")


Generating 4200 train images...


100%|████████████████████████████████████████████████████████████████████████████| 4200/4200 [1:35:41<00:00,  1.37s/it]


Generating 1200 val images...


100%|██████████████████████████████████████████████████████████████████████████████| 1200/1200 [27:10<00:00,  1.36s/it]


Generating 600 test images...


100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [13:40<00:00,  1.37s/it]


Dataset v5 Ready! (Realistic Table Version)


In [1]:
import torch
torch.cuda.is_available(), torch.cuda.get_device_name(0)


(True, 'NVIDIA GeForce RTX 3070 Laptop GPU')

In [2]:
import torch
x = torch.rand(10).cuda()
y = torch.rand(10).cuda()
z = x + y
print(z)


tensor([0.7573, 0.2473, 0.9459, 0.8966, 0.9941, 1.4526, 0.8472, 0.7592, 0.2600,
        0.8623], device='cuda:0')


In [6]:
import os

print(os.path.abspath("synthetic_shapes_v5"))


C:\Users\hp omen 4 pro\synthetic_shapes_v5


In [8]:
yaml_text = """
path: C:/Users/hp omen 4 pro/synthetic_shapes_v5

train: C:/Users/hp omen 4 pro/synthetic_shapes_v5/images/train
val:   C:/Users/hp omen 4 pro/synthetic_shapes_v5/images/val
test:  C:/Users/hp omen 4 pro/synthetic_shapes_v5/images/test

names:
  0: circle_white
  1: circle_black
  2: circle_red
  3: circle_green
  4: circle_blue

  5: square_white
  6: square_black
  7: square_red
  8: square_green
  9: square_blue

  10: triangle_white
  11: triangle_black
  12: triangle_red
  13: triangle_green
  14: triangle_blue

  15: octagon_white
  16: octagon_black
  17: octagon_red
  18: octagon_green
  19: octagon_blue

  20: star_white
  21: star_black
  22: star_red
  23: star_green
  24: star_blue
"""

with open("synthetic_shapes_v5.yaml", "w") as f:
    f.write(yaml_text)

print("YAML file saved!")


YAML file saved!


In [1]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

model.train(
    data="synthetic_shapes_v5.yaml",
    epochs=20,
    imgsz=640,
    batch=4,
    workers=1,
    device=0,
    amp=False
)


New https://pypi.org/project/ultralytics/8.3.231 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.58  Python-3.9.21 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=synthetic_shapes_v5.yaml, epochs=20, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=1, project=None, name=train14, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show

train: Scanning C:\Users\hp omen 4 pro\synthetic_shapes_v5\labels\train.cache... 4200 images, 0 backgrounds, 0 corrupt:
val: Scanning C:\Users\hp omen 4 pro\synthetic_shapes_v5\labels\val.cache... 1200 images, 0 backgrounds, 0 corrupt: 100


Plotting labels to runs\detect\train14\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000345, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 1 dataloader workers
Logging results to runs\detect\train14
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/20      1.16G     0.9842       3.04      1.324         47        640: 100%|██████████| 1050/1050 [06:01<00:00,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 150/150 [00:


                   all       1200       4730      0.851      0.771      0.851      0.685

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       1200       4730      0.989      0.949      0.979      0.871



20 epochs completed in 2.040 hours.
Optimizer stripped from runs\detect\train14\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train14\weights\best.pt, 6.3MB

Validating runs\detect\train14\weights\best.pt...
Ultralytics 8.3.58  Python-3.9.21 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
Model summary (fused): 168 layers, 3,010,523 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 150/150 [00:


                   all       1200       4730      0.989      0.949      0.979      0.871
          circle_white        172        179      0.993      0.972      0.985       0.91
          circle_black        185        199       0.99       0.95      0.984      0.887
            circle_red        192        203      0.992       0.97      0.977      0.896
          circle_green        170        184      0.999      0.995      0.995      0.894
           circle_blue        170        182          1      0.981      0.986        0.9
          square_white        177        186      0.989      0.963       0.99      0.905
          square_black        154        167      0.981      0.976      0.988      0.904
            square_red        189        210      0.994      0.971      0.995      0.907
          square_green        177        190      0.985      0.968      0.986      0.911
           square_blue        212        230      0.989      0.974      0.994      0.916
        triangle_whit

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000026F43420910>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.04104

In [3]:
best_model = YOLO("runs/detect/train14/weights/best.pt")
best_model.val()


Ultralytics 8.3.58  Python-3.9.21 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
Model summary (fused): 168 layers, 3,010,523 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\hp omen 4 pro\synthetic_shapes_v5\labels\val.cache... 1200 images, 0 backgrounds, 0 corrupt: 100
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 75/75 [00:39


                   all       1200       4730      0.989      0.949      0.979      0.871
          circle_white        172        179      0.993      0.972      0.985       0.91
          circle_black        185        199       0.99       0.95      0.984      0.888
            circle_red        192        203      0.992       0.97      0.977      0.896
          circle_green        170        184      0.999      0.995      0.995      0.894
           circle_blue        170        182          1      0.981      0.986        0.9
          square_white        177        186      0.989      0.963       0.99      0.905
          square_black        154        167      0.981      0.976      0.988      0.904
            square_red        189        210      0.994      0.971      0.995      0.907
          square_green        177        190      0.985      0.968      0.986      0.911
           square_blue        212        230      0.989      0.974      0.994      0.916
        triangle_whit

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000026FB98F28B0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.04104

In [9]:
best_model("sample.bmp", save=True, imgsz=640)



image 1/1 C:\Users\hp omen 4 pro\sample.bmp: 544x640 1 octagon_red, 1 octagon_green, 1 octagon_blue, 42.1ms
Speed: 12.0ms preprocess, 42.1ms inference, 5.9ms postprocess per image at shape (1, 3, 544, 640)
Results saved to runs\detect\predict3


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'circle_white', 1: 'circle_black', 2: 'circle_red', 3: 'circle_green', 4: 'circle_blue', 5: 'square_white', 6: 'square_black', 7: 'square_red', 8: 'square_green', 9: 'square_blue', 10: 'triangle_white', 11: 'triangle_black', 12: 'triangle_red', 13: 'triangle_green', 14: 'triangle_blue', 15: 'octagon_white', 16: 'octagon_black', 17: 'octagon_red', 18: 'octagon_green', 19: 'octagon_blue', 20: 'star_white', 21: 'star_black', 22: 'star_red', 23: 'star_green', 24: 'star_blue'}
 obb: None
 orig_img: array([[[167, 206, 214],
         [167, 191, 177],
         [171, 180, 177],
         ...,
         [154, 146, 145],
         [148, 150, 145],
         [148, 150, 145]],
 
        [[167, 193, 191],
         [167, 179, 160],
         [171, 167, 160],
         ...,
         [154, 149, 146],
         [148, 146, 146],
         [148, 146, 146]],
 
    

In [11]:
from ultralytics import YOLO
import glob
import os

# 1) Load your model
model = YOLO("runs/detect/train14/weights/best.pt")   # ← change to your model path

# 2) Folder with BMP files
folder = r"C:\Users\hp omen 4 pro\Aruco pictures AI"

# 3) Get list of BMP files
bmp_files = glob.glob(folder + "/*.bmp")

print("Found", len(bmp_files), "BMP images.")

# 4) Create output folder
output_dir = os.path.join(folder, "predictions")
os.makedirs(output_dir, exist_ok=True)

# 5) Run YOLO on each image
for img_path in bmp_files:
    print("Predicting:", img_path)

    model.predict(
        source=img_path,
        save=True,
        save_txt=True,
        save_conf=True,
        imgsz=640,
        project=output_dir,
        name="results",
        exist_ok=True
    )

print("\nDone! Check predictions folder:")
print(output_dir)


Found 14 BMP images.
Predicting: C:\Users\hp omen 4 pro\Aruco pictures AI\Image__2025-09-09__15-24-37.bmp

image 1/1 C:\Users\hp omen 4 pro\Aruco pictures AI\Image__2025-09-09__15-24-37.bmp: 544x640 1 circle_red, 6 square_whites, 1 square_black, 1 triangle_white, 43.6ms
Speed: 13.0ms preprocess, 43.6ms inference, 7.0ms postprocess per image at shape (1, 3, 544, 640)
Results saved to C:\Users\hp omen 4 pro\Aruco pictures AI\predictions\results
1 label saved to C:\Users\hp omen 4 pro\Aruco pictures AI\predictions\results\labels
Predicting: C:\Users\hp omen 4 pro\Aruco pictures AI\Image__2025-09-09__15-35-28.bmp

image 1/1 C:\Users\hp omen 4 pro\Aruco pictures AI\Image__2025-09-09__15-35-28.bmp: 544x640 1 circle_white, 1 square_white, 1 square_green, 1 star_white, 38.4ms
Speed: 11.1ms preprocess, 38.4ms inference, 9.0ms postprocess per image at shape (1, 3, 544, 640)
Results saved to C:\Users\hp omen 4 pro\Aruco pictures AI\predictions\results
2 labels saved to C:\Users\hp omen 4 pro\Aruc